*Теоретический материал:* https://youtu.be/cbfsRIch4V0?t=77

In [ ]:
import tensorflow as tf
import datetime,os
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import seaborn as sns
sns.set_style('darkgrid')
import matplotlib.pyplot as plt

# LIGHT

## LIGHT 1

Создаем генератор, из которого будем создавать x и y

In [ ]:
random_gen = tf.random.Generator.from_seed(1)

In [ ]:
random_gen

In [ ]:
for _ in range(4):
  print(random_gen.normal(shape=[2, 3]))

Создаем функцию генератора, которая будет генерить x и y в датасете

In [ ]:
def gen():
  x = random_gen.normal(shape=[2, 3])
  y = random_gen.normal(shape=[])
  yield x, y

Создаем датасет, указываем генератор, описываем структуру

In [ ]:
random_DS = tf.data.Dataset.from_generator(gen,
                                           output_signature = (
                                               tf.TensorSpec(shape=(2, 3), dtype=tf.float32),
                                               tf.TensorSpec(shape=(), dtype=tf.float32)
                                           )
                                           )

Проверяем

In [ ]:
list(random_DS.take(1))

## LIGHT 2


Определяем модель

In [ ]:
class myModel(object):
  def __init__(self):
    self.W = tf.Variable(5.0)
    self.b = tf.Variable(10.0)

  def __call__(self, x):
    return tf.cos(self.W * x + self.b)

model = myModel()

Определяем ошибку

In [ ]:
def loss(target_y, predicted_y):
  return tf.reduce_mean(tf.square(target_y - predicted_y))

задаем целевые значения, входящие значения

In [ ]:
TRUE_W = 6.0
TRUE_b = 2.0
NUM_EXAMPLES = 1000

inputs  = tf.random.normal(shape=[NUM_EXAMPLES])
outputs = tf.cos(inputs * TRUE_W + TRUE_b)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(14,7))
plt.scatter(inputs, outputs, c='b')
plt.scatter(inputs, model(inputs), c='r')
plt.show()

print('Current loss: %1.6f' % loss(model(inputs), outputs).numpy())

In [ ]:
def train(model, inputs, outputs, learning_rate):
  with tf.GradientTape() as t:
    current_loss = loss(outputs, model(inputs))
    dW, db = t.gradient(current_loss, [model.W, model.b])
    model.W.assign_sub(learning_rate * dW)
    model.b.assign_sub(learning_rate * db)

In [ ]:
!rm -R /tmp/mylogs/eager

In [ ]:
!ls '/tmp/mylogs/eager'

## LIGHT 3

In [ ]:
path = '/tmp/mylogs/eager/'

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
loss_log_dir = path + current_time + '/data'
loss_summary_writer = tf.summary.create_file_writer(loss_log_dir)

model = myModel()

with loss_summary_writer.as_default():
  Ws, bs = [], []
  epoch = 0
  while epoch<1e15:
    Ws.append(model.W.numpy())
    bs.append(model.b.numpy())
    current_loss = loss(outputs, model(inputs))

    train(model, inputs, outputs, learning_rate=0.001)
    print('Epoch %2d: W=%1.2f b=%1.2f, loss=%2.5f' %
          (epoch, Ws[-1], bs[-1], current_loss))

    tf.summary.scalar("current_loss", current_loss, step=epoch)
    tf.summary.scalar("W", Ws[-1], step=epoch)
    tf.summary.scalar("b", bs[-1], step=epoch)
    epoch+=1
    loss_summary_writer.flush()


plt.figure(figsize=(14,7))
plt.plot(range(epoch), Ws, 'r',
         range(epoch), bs, 'b')
plt.plot([TRUE_W] * epoch, 'r--',
         [TRUE_b] * epoch, 'b--')
plt.legend(['W', 'b', 'True W', 'True b'])
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(14,7))
plt.scatter(inputs, outputs, c='b')
plt.scatter(inputs, model(inputs), c='r')
plt.show()

print('Current loss: %1.6f' % loss(model(inputs), outputs).numpy())

In [ ]:
%load_ext tensorboard

In [ ]:
!ls /tmp/mylogs/eager

In [ ]:
%tensorboard --logdir '/tmp/mylogs/eager/'

# PRO

In [ ]:
# создаем тензор с х
x = tf.random.uniform([100, 3])
x

In [ ]:
# создаем тезор с у (в данном случае аналог биас)
y =tf.concat([tf.ones([50,1]), tf.zeros([50,1])], axis=0)

In [ ]:
y

In [ ]:
# создадим тезор с начальными весами
w = tf.random.normal([3, 1], stddev=0.3)

In [ ]:
w

In [ ]:
# определим модель, в которой изменяться будут только веса, а возвращаться перемноженные входные данные на веса + биас
class myModel(object):
  def __init__(self, w):
    self.W = tf.Variable(w)
    # self.b = tf.Variable(10.0)

  def __call__(self, x, y):
    return tf.add(tf.matmul(x, self.W), y)

model = myModel(w)

In [ ]:
model(x,y)

In [ ]:
# зададим целевые данные
y_true = tf.random.uniform([100, 1])

In [ ]:
y_true

In [ ]:
# определим ошибку мае
def loss(target_y, predicted_y):
  return tf.reduce_mean(tf.abs(target_y - predicted_y))

In [ ]:
# определим функцию обучения с градиентным спуском и применением оптимизатора
opt = tf.optimizers.Adam()

def train(model, x, y, outputs):
    with tf.GradientTape() as tape:
      cur_loss = loss(y_true, model(x,y))
    vars = [model.W]
    grads = tape.gradient(cur_loss, vars)
    # processed_grads = [process_gradient(g) for g in grads]

# Ask the optimizer to apply the processed gradients.
    opt.apply_gradients(zip(grads, vars))

In [ ]:
!rm -R /tmp/mylogs/eager

In [ ]:
!ls '/tmp/mylogs/eager'

In [ ]:
# создаем файл для записи логов для последующей визуализации и обучаем модель
path = '/tmp/mylogs/eager/'

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
loss_log_dir = path + current_time + '/data'
loss_summary_writer = tf.summary.create_file_writer(loss_log_dir)

model = myModel(w)

with loss_summary_writer.as_default():
  
  Ws = []
  epoch = 0
  while epoch<1000:

    Ws.append(model.W.numpy())
    current_loss = loss(y_true, model(x,y))
    train(model, x, y, y_true)
    print('Epoch %2d:  loss=%2.5f' %
          (epoch, current_loss))
    print(Ws[-1])
    tf.summary.scalar("current_loss", current_loss, step=epoch)
    epoch+=1
    loss_summary_writer.flush()

In [ ]:
%load_ext tensorboard

In [ ]:
!ls /tmp/mylogs/eager

In [ ]:
%tensorboard --logdir '/tmp/mylogs/eager/'